In [52]:
import pandas as pd
import numpy as np
# Get some time series data
df = pd.read_csv("height2.csv")
df = df[['height', 'temp', 'rainfallh']]
#df.head()
df.dropna()

,height,temp,rainfallh
0,4.63,45.0,0.00
1,4.66,44.5,0.00
2,4.70,44.2,0.00
3,4.78,44.0,0.00
4,4.82,43.5,0.00
5,4.86,42.8,0.00
6,4.90,41.9,0.00
7,4.94,41.9,0.00
8,4.97,42.7,0.00
9,5.03,43.8,0.00


In [53]:
# Put your inputs into a single list


def get_split(dataset):
    #print(dataset.drop('height',1))
    return dataset['height'], dataset.drop('height',1)
height2, predictors = get_split(df)

#print(predictors[rainfallh].head())



df['single_input_vector'] = predictors.apply(tuple, axis=1).apply(list)
# Double-encapsulate list so that you can sum it in the next step and keep time steps as separate elements
df['single_input_vector'] = df.single_input_vector.apply(lambda x: [list(x)])
# Use .cumsum() to include previous row vectors in the current row list of vectors
df['cumulative_input_vectors'] = df.single_input_vector.cumsum()
print(height.head())
#df['output_vector'] = df.apply(tuple, axis=1).apply(list)


0    4.63
1    4.66
2    4.70
3    4.78
4    4.82
Name: height, dtype: float64


In [54]:
from keras.preprocessing.sequence import pad_sequences

max_sequence_length = df.cumulative_input_vectors.apply(len).max()
# Save it as a list   
padded_sequences = pad_sequences(df.cumulative_input_vectors.tolist(), max_sequence_length).tolist()
df['padded_input_vectors'] = pd.Series(padded_sequences).apply(np.asarray)

In [75]:
print(len(df))
print(X_train_init.shape)
s = np.hstack(X_train_init)
#fin = s.reshape(3535,3535,2)
y_train = np.hstack(np.asarray(height))

3535
(3535,)


In [66]:
# Extract your training data
X_train_init = np.asarray(df.padded_input_vectors)
# Use hstack to and reshape to make the inputs a 3d vector
X_train = np.hstack(X_train_init).reshape(len(df),max_sequence_length, 1)
y_train = np.hstack(np.asarray(height)).reshape(len(df),len(height))


ValueError: cannot reshape array of size 24992450 into shape (3535,3535,1)

In [148]:
#df['output_vector'].head()
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys
def build_model(layers):
    print(layers)
    model = Sequential()

    model.add(LSTM(
        input_dim=layers[0],
        output_dim=1,
        return_sequences=True))
    model.add(Dropout(0.2))

    model.add(LSTM(
        100,
        return_sequences=False))
    model.add(Dropout(0.2))

    model.add(Dense(
        1))
    model.add(Activation("linear"))

    
    model.compile(loss="mse", optimizer="rmsprop")
    #print("> Compilation Time : ", time.time() - start)
    return model
input_length = fin.shape[1]
input_dim = fin.shape[2]
# Output dimensions is the shape of a single output vector
# In this case it's just 1, but it could be more
output_dim = len(y_train)

In [149]:
model = build_model([2, 3535,3535,2])
print(model.summary())

[2, 3535, 3535, 2]


//anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:17: UserWarning: The `input_dim` and `input_length` arguments in recurrent layers are deprecated. Use `input_shape` instead.
//anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:17: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(return_sequences=True, input_shape=(None, 2), units=1)`


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_52 (LSTM)               (None, None, 1)           16        
_________________________________________________________________
dropout_31 (Dropout)         (None, None, 1)           0         
_________________________________________________________________
lstm_53 (LSTM)               (None, 100)               40800     
_________________________________________________________________
dropout_32 (Dropout)         (None, 100)               0         
_________________________________________________________________
dense_15 (Dense)             (None, 1)                 101       
_________________________________________________________________
activation_15 (Activation)   (None, 1)                 0         
Total params: 40,917.0
Trainable params: 40,917.0
Non-trainable params: 0.0
_________________________________________________________________


In [ ]:

#load_data("height.csv", 20 ,True)
model = build_model([2, 3535, 3535, 2])
print(model.summary())
model.fit(
    fin,
    y_train,
    batch_size=512,
    nb_epoch=2,
    validation_split=0.05)



[2, 3535, 3535, 2]


//anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:17: UserWarning: The `input_dim` and `input_length` arguments in recurrent layers are deprecated. Use `input_shape` instead.
//anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:17: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(return_sequences=True, input_shape=(None, 2), units=1)`


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_54 (LSTM)               (None, None, 1)           16        
_________________________________________________________________
dropout_33 (Dropout)         (None, None, 1)           0         
_________________________________________________________________
lstm_55 (LSTM)               (None, 100)               40800     
_________________________________________________________________
dropout_34 (Dropout)         (None, 100)               0         
_________________________________________________________________
dense_16 (Dense)             (None, 1)                 101       
_________________________________________________________________
activation_16 (Activation)   (None, 1)                 0         
Total params: 40,917.0
Trainable params: 40,917.0
Non-trainable params: 0.0
_________________________________________________________________


//anaconda/lib/python3.6/site-packages/keras/models.py:826: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 3358 samples, validate on 177 samples
Epoch 1/2
3358/3358 [==============================] - 1596s - loss: 11.3071 - val_loss: 14.1026
Epoch 2/2


In [135]:
def predict_sequences_multiple(model, data, window_size, prediction_len):
    #Predict sequence of 50 steps before shifting prediction run forward by 50 steps
    prediction_seqs = []
    for i in range(int(len(data)/prediction_len)):
        curr_frame = data[i*prediction_len]
        predicted = []
        for j in range(prediction_len):
            predicted.append(model.predict(curr_frame[newaxis,:,:])[0,0])
            curr_frame = curr_frame[1:]
            curr_frame = np.insert(curr_frame, [window_size-1], predicted[-1], axis=0)
        prediction_seqs.append(predicted)
    return prediction_seqs

In [134]:
model.save('my_model.h5')

In [146]:
def read_pdf(path):
    df = pd.read_csv(path)
    df = df[['height', 'temp', 'rainfallh']]
    #df.head()
    df.dropna()
    #X_test, actual = get_split(df)
    
    # Save it as a list   
    
    
    df['single_input_vector'] = predictors.apply(tuple, axis=1).apply(list)
    # Double-encapsulate list so that you can sum it in the next step and keep time steps as separate elements
    df['single_input_vector'] = df.single_input_vector.apply(lambda x: [list(x)])
    df['cumulative_input_vectors'] = df.single_input_vector.cumsum()
    max_sequence_length = df.cumulative_input_vectors.apply(len).max()
    padded_sequences = pad_sequences(df.cumulative_input_vectors.tolist(), max_sequence_length).tolist()
    df['padded_input_vectors'] = pd.Series(padded_sequences).apply(np.asarray)
    print(len(df))
    X_train_init = np.asarray(df.padded_input_vectors)
    print(X_train_init.shape)
    s = np.hstack(X_train_init)
    fin = s.reshape(327,327,2)
    y_train = np.hstack(np.asarray(height))
    return fin, y_train
X_test, y =read_pdf("height3.csv")

327
(327,)


In [147]:
model.predict(X_test)

ValueError: Error when checking : expected lstm_50_input to have shape (None, 3535, 2) but got array with shape (327, 327, 2)